# Implementação 1: Toy Example

Esse notebook apresenta a primeira implementação do Toy Example proposto.
Sua proposta segue os seguintes objetivos:
* A implementação não seguirá padrões sugeridos nos campos das dimensões da IAR.
* Esse notebook servirá de checagem para análise da eficiência e corretude da checklist proposta pelo NIAR.

## Processamento dos dados

Os dados são de origem do Datasus, contendo informações das internações nos períodos de 2022 à novembro de 2025 (dados mais recentes quando feito esse Toy Example).

Devido ao tamanho dos dados brutos, não se encontram na página do Github. No entanto, a tabela final (após todo o pré processamento) se encontra em "Tabela_lag_final.zip". Apenas essa tabela é necessária para a execução do notebook. Para tal, descomprima o arquivo zip e coloque o arquivo "Tabela_lag_final.csv" (o único arquivo csv) dentro do diretório "SIH_Dados/".

O download dos dados foi feito por conexão ftp com o servidor disponibilizado pelo datasus: ftp.datasus.gov.br. Especificamente, os dados se encontram no diretório "/dissemin/publicos/SIHSUS/200801_/Dados/". Foram usados todos os arquivos que começam com "RD" e contém "22", "23", "24", "25", no quinto e sexto caractére (quando escrito esse notebook, exclui-se os dados de dezembro de 2025 e os dados de novembro de 2025 dos estados "AC" e "RR", pois não foram divulgados ainda).

A fim de evitar refazer grandes cálculos toda vez que esse notebook é executado, o processamento dos dados - filtragem de colunas de interesse e cálculo de novas colunas - foi feito previamente. No diretório SIH_Dados, encontram-se os scripts usados para cada etapa do processamento:
* dbc_dbf.py e dbf_csv.py: Transformam os arquivos de dados do formato dbc para dbf e dbf para csv, respectivamente.
* filtrar.py: Lê os arquivos csv, filtram as colunas (e o "target" de interesse, DIAG_PRIN = "J...") e salvam em novos csv.
* tabelacao_agregado.py: Lê os novos arquivos csv, e calcula os valores agregados dos dados, agrupando-os por mes/ano e hospital. A tabela resultante é salva em "Tabela_Agregada.csv".
* tabelacao_lag_final.py: Lê a tabelacao do passo anterior e calcula as variaveis "lag", usadas no modelo para predicao. A tabela, com todos os atributos que serão usados para o modelo final, é salva em um último csv: "Tabela_lag_Final.csv".

O intuito de separar cada processo em seu próprio script é devido a robustez contra a possibilidade de erros ou mudanças, permitindo que alterações sejam feitas sem a necessidade de recomeçar o processo do zero. Além disso, cada script demora vários minutos, alguns até passando de uma hora, para terminar. Juntar todos os scripts em um resultaria em um programa que terminaria após várias horas.

### Etapa 1: Inicialização
Import's usados, e leitura dos dados 

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import sklearn as sk
from sklearn.metrics import mean_absolute_error, mean_squared_error


df = pd.read_csv("./SIH_Dados/Tabela_lag_Final.csv")

### Etapa 2: Limpagem e separação dos dados
Retirada de linhas com dados nulos e separação dos dados de treinamento, validação e teste.

In [2]:
df.dropna(inplace=True)


X_Train = df[(df.year >= 2022) & (df.year <= 2024)]
X_Train = X_Train[(X_Train.year < 2024) | (X_Train.month <= 6)].reset_index(drop=True)
Y_Train = X_Train["J_count"]
X_Train.drop(columns=["J_count", "region"], inplace=True)

X_Valid = df[(df.year == 2024) & (df.month >= 7)].reset_index(drop=True)
Y_Valid = X_Valid["J_count"]
X_Valid.drop(columns=["J_count", "region"], inplace=True)

X_Test = df[df.year == 2025].reset_index(drop=True)
Y_Test = X_Test["J_count"]
X_Test.drop(columns=["J_count", "region"], inplace=True)


### Etapa 3: Modelo Base
Testes usando modelo de Regressão Linear, implementada pelo scikit learn

In [3]:
model = sk.linear_model.LinearRegression().fit(X_Train, Y_Train)

val_resul = model.predict(X_Valid)
test_resul = model.predict(X_Test)

### Etapa 4: Modelo LightGBM

Testes usando modelo com objetivo Poisson, implementada pelo lightgbm

In [4]:
seed = 778

lgb_train = lgb.Dataset(X_Train, Y_Train)
lgb_valid = lgb.Dataset(X_Valid, Y_Valid)


model = lgb.train(params={"objective": "poisson"}, train_set=lgb_train, valid_sets=lgb_valid, callbacks=[lgb.early_stopping(stopping_rounds=5)])

res_pred = model.predict(X_Test, num_iteration=model.best_iteration)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012316 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24420
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 105
[LightGBM] [Info] Start training from score 3.504180
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's poisson: -91.2195


### Etapa 5: Métricas

Comparação dos resultados de cada modelo

In [5]:
df_resultados = X_Test[["month", "CNES", "hospital_porte"]]
df_resultados["year"] = 2025
df_resultados = df_resultados[["year", "month", "CNES", "hospital_porte"]]
df_resultados["Valor Real"] = Y_Test
df_resultados["Base resultados"] = test_resul
df_resultados["LightGBM resultados"] = res_pred


base_mae = mean_absolute_error(y_true=Y_Test, y_pred=test_resul)
lgb_mae = mean_absolute_error(y_true=Y_Test, y_pred=res_pred)


base_rmse = mean_squared_error(y_true=Y_Test, y_pred=test_resul)
lgb_rmse = mean_squared_error(y_true=Y_Test, y_pred=res_pred)

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

base_smape = smape(y_true=Y_Test, y_pred=test_resul)
lgb_smape = smape(y_true=Y_Test, y_pred=res_pred)

print(df_resultados)
print()
print("------------------------------ RESULTADOS GLOBAIS ------------------------------")
print("\t\t\t|\t\t\t\t|")
print("\tMetrica\t\t|\t\tBase\t\t|\tLightGBM")
print("------------------------+-------------------------------+-----------------------")
print("MAE\t\t\t|", base_mae, "\t\t|", lgb_mae)
print("\t\t\t|\t\t\t\t|")
print("RMSE\t\t\t|", base_rmse, "\t\t|", lgb_rmse)
print("\t\t\t|\t\t\t\t|")
print("SMAPE\t\t\t|", base_smape, "\t\t|", lgb_smape)
print("\t\t\t|\t\t\t\t|")



       year  month     CNES  hospital_porte  Valor Real  Base resultados  \
0      2025      1  2000385               3          21        24.028848   
1      2025      1    10456               3         133       117.093101   
2      2025      1    10499               3         190       101.101042   
3      2025      1    10502               3          97        79.283146   
4      2025      1  6876617               3          64        40.103717   
...     ...    ...      ...             ...         ...              ...   
33972  2025     11  2006197               1          10         6.419394   
33973  2025     11  2010631               2          12        33.744214   
33974  2025     11  5616298               3          39        42.531451   
33975  2025     11  2010151               2          29        26.444760   
33976  2025     11  2722399               1          30        10.870715   

       LightGBM resultados  
0                19.295646  
1               175.808902  


In [6]:

for i in range (0, 4):
    print("")
    df_porte = df_resultados[df_resultados.hospital_porte == i]
    y_t = df_porte["Valor Real"]
    y_base = df_porte["Base resultados"]
    y_lgb = df_porte["LightGBM resultados"]
    
    base_mae = mean_absolute_error(y_true=y_t, y_pred=y_base)
    lgb_mae = mean_absolute_error(y_true=y_t, y_pred=y_lgb)

    base_rmse = mean_squared_error(y_true=y_t, y_pred=y_base)
    lgb_rmse = mean_squared_error(y_true=y_t, y_pred=y_lgb)

    base_smape = smape(y_true=y_t, y_pred=y_base)
    lgb_smape = smape(y_true=y_t, y_pred=y_lgb)

    t = "pequeno"
    if (i == 1):
        t = "medio"
    if (i == 2):
        t = "grande"
    if (i == 3):
        t = "extra"
    
    print("--------------------------- RESULTADOS: Porte", t, "---------------------------")
    print("\t\t\t|\t\t\t\t|")
    print("\tMetrica\t\t|\t\tBase\t\t|\tLightGBM")
    print("------------------------+-------------------------------+-----------------------")
    print("MAE\t\t\t|", base_mae, "\t\t|", lgb_mae)
    print("\t\t\t|\t\t\t\t|")
    print("RMSE\t\t\t|", base_rmse, "\t\t|", lgb_rmse)
    print("\t\t\t|\t\t\t\t|")
    print("SMAPE\t\t\t|", base_smape, "\t\t|", lgb_smape)
    print("\t\t\t|\t\t\t\t|")
    print()


--------------------------- RESULTADOS: Porte pequeno ---------------------------
			|				|
	Metrica		|		Base		|	LightGBM
------------------------+-------------------------------+-----------------------
MAE			| 6.449604843853462 		| 5.091679644665822
			|				|
RMSE			| 78.60482425000741 		| 45.551840595623496
			|				|
SMAPE			| 88.49473901464185 		| 59.00296331511675
			|				|


--------------------------- RESULTADOS: Porte medio ---------------------------
			|				|
	Metrica		|		Base		|	LightGBM
------------------------+-------------------------------+-----------------------
MAE			| 8.44737856338532 		| 7.862024854876532
			|				|
RMSE			| 144.23810253669944 		| 122.54374996370794
			|				|
SMAPE			| 54.98238815528808 		| 47.97431775231376
			|				|


--------------------------- RESULTADOS: Porte grande ---------------------------
			|				|
	Metrica		|		Base		|	LightGBM
------------------------+-------------------------------+-----------------------
MAE			| 12.417219595473982 		| 11